In [186]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 500
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colors import LogNorm
from matplotlib.patches import FancyArrowPatch
from astropy.visualization import SqrtStretch, LinearStretch, LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import pandas as pd
import re
from astropy.io import fits
import os
from scipy.ndimage import map_coordinates
from scipy.stats import binned_statistic
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

from matplotlib.lines import Line2D
os.chdir("/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling")

In [187]:
# Model Opening
model_path = 'Sersic/SIE'
model_ver = 'SIE_POS'
lens_name = f'"{model_path}"'

if 'POS+FLUX' in model_ver:
    constraint = 'pos_flux'
elif 'POS' in model_ver:
    constraint = 'pos'


# Load the data
with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
    opt_result = file.readlines()

opt_result

['------------------------------------------\n',
 'optimize ndim=13\n',
 'run 1: 77 lens models calculated\n',
 'chi^2 = 1.840558e+02  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 1.840558e+02 1.840558e+02 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.579333e+02  2.080932e+01  2.075843e+01  1.261772e-01 -4.523281e+00  0.000000e+00  0.000000e+00 \n',
 'lens   sers    0.2613  7.970685e+10  2.080932e+01  2.075843e+01  1.917400e-01  9.715570e+01  5.188100e+01  1.442400e+00 \n',
 'lens   sers    0.2613  9.953921e+09  2.080932e+01  2.075843e+01  5.818500e-01  8.090880e+01  5.979260e+00  5.737000e-01 \n',
 'point  1.0000  2.081230e+01  2.077865e+01 \n',
 '------------------------------------------\n',
 '------------------------------------------\n',
 'optimize ndim=13

In [188]:
# Find the last line with 'optimize' in it
last_optimize_index = None
for idx in range(len(opt_result) - 1, -1, -1):
    if 'optimize' in opt_result[idx]:
        last_optimize_index = idx
        last_optimize_line = opt_result[idx]
        break
if last_optimize_index is None:
    raise ValueError("No line with 'optimize' found in the file.")

print(f"Last 'optimize' line found at index: {last_optimize_index}")

# Extract everything after the last 'optimize' line
opt_result = opt_result[last_optimize_index + 1:]

opt_result


Last 'optimize' line found at index: 97


['run 7: 30 lens models calculated\n',
 'chi^2 = 1.839777e+02  [N_data(extend): 0]\n',
 ' extend     : 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' point no 1 : 1.839777e+02 1.839777e+02 0.000000e+00 0.000000e+00 0.000000e+00\n',
 ' lens prior : 0.000000e+00\n',
 ' map prior  : 0.000000e+00\n',
 '\n',
 'omega = 0.3000  lambda = 0.7000  weos = -1.0000  hubble = 0.7000\n',
 '\n',
 'lens   sie     0.2613  1.579333e+02  2.080932e+01  2.075843e+01  1.261772e-01 -4.523281e+00  0.000000e+00  0.000000e+00 \n',
 'lens   sers    0.2613  7.970685e+10  2.080932e+01  2.075843e+01  1.917400e-01  9.715570e+01  5.188100e+01  1.442400e+00 \n',
 'lens   sers    0.2613  9.487091e+09  2.080932e+01  2.075843e+01  5.818500e-01  8.090880e+01  5.979260e+00  5.737000e-01 \n',
 'point  1.0000  2.081230e+01  2.077867e+01 \n',
 '------------------------------------------\n']

In [189]:
# Count the number of lines that start with 'lens'
lens_count = sum(1 for line in opt_result if line.startswith('lens'))
print(f"Number of 'lens' lines: {lens_count}")

# Initialize a dictionary to hold the lens parameters
lens_params_dict = {}

# Extract the lens parameters
lens_params = []
for line in opt_result:
    if line.startswith('lens'):
        # Extract the line and split it into parts
        parts = re.split(r'\s+', line.strip())
        print(parts)
        # Extract the lens name and parameters
        lens_name = parts[1]
        params = [float(x) for x in parts[2:]]

        # Store the parameters in the dictionary, allowing duplicate names as separate entries
        key = lens_name
        count = 1
        while key in lens_params_dict:
            key = f"{lens_name}_{count}"
            count += 1
        lens_params_dict[key] = params
        lens_params.append((key, params))

# Remove the first lens parameter
if lens_params:
    for i in range(len(lens_params)):
        lens_name, params = lens_params[i]
        lens_params_dict[lens_name] = params[1:]


# Number of len profiles
num_lens_profiles = len(lens_params_dict)
print(f"Number of lens profiles: {num_lens_profiles}")
    
lens_params_dict

Number of 'lens' lines: 3
['lens', 'sie', '0.2613', '1.579333e+02', '2.080932e+01', '2.075843e+01', '1.261772e-01', '-4.523281e+00', '0.000000e+00', '0.000000e+00']
['lens', 'sers', '0.2613', '7.970685e+10', '2.080932e+01', '2.075843e+01', '1.917400e-01', '9.715570e+01', '5.188100e+01', '1.442400e+00']
['lens', 'sers', '0.2613', '9.487091e+09', '2.080932e+01', '2.075843e+01', '5.818500e-01', '8.090880e+01', '5.979260e+00', '5.737000e-01']
Number of lens profiles: 3


{'sie': [157.9333, 20.80932, 20.75843, 0.1261772, -4.523281, 0.0, 0.0],
 'sers': [79706850000.0, 20.80932, 20.75843, 0.19174, 97.1557, 51.881, 1.4424],
 'sers_1': [9487091000.0,
  20.80932,
  20.75843,
  0.58185,
  80.9088,
  5.97926,
  0.5737]}

In [190]:
# Initialize a dictionary to hold the lens parameters
set_point_dict = {}

# Extract the lens parameters
set_points = []
for line in opt_result:
    if line.startswith('point'):
        # Extract the line and split it into parts
        parts = re.split(r'\s+', line.strip())
        print(parts)

set_point_values = [float(x) for x in parts[1:]]

for i in range(len(set_point_values)):
    set_point_values[i] = set_point_values[i] -20.78

set_point_values

['point', '1.0000', '2.081230e+01', '2.077867e+01']


[-19.78, 0.03229999999999933, -0.0013299999999993872]

In [191]:
# Use generic column names: param1, param2, ...
df = pd.DataFrame()
rows = []
max_param_len = 0

for lens_name, params in lens_params_dict.items():
    row = {'Lens Name': lens_name}
    for i, val in enumerate(params):
        row[f'param{i+1}'] = val
    rows.append(row)
    if len(params) > max_param_len:
        max_param_len = len(params)

columns = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len)]
df = pd.DataFrame(rows, columns=columns)

# df['Lens Name'] = df['Lens Name'].apply(lambda x: f'"{x}"')

In [192]:
for i in range(len(df)):
    if df['Lens Name'][i] == 'sers_1':
        df['Lens Name'][i] = 'sers'

/var/folders/tm/964hg0yn70x0ddsqcj4bsf8r0000gn/T/ipykernel_81692/2992054562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lens Name'][i] = 'sers'


In [193]:
df['param2'] = df['param2'] - 20.78
df['param3'] = df['param3'] - 20.78

In [194]:
with open('write_lens' + '.py', 'r') as file:
    py = file.readlines()

py

['#!/usr/bin/env python\n',
 'import glafic\n',
 '\n',
 "glafic.init(0.3, 0.7, -1.0, 0.7, 'Sersic/SIE/SIE_POS', -0.78, -0.78, 0.78, 0.78, 0.012, 0.012, 1, verb = 0)\n",
 '\n',
 "glafic.set_secondary('chi2_splane 1', verb = 0)\n",
 "glafic.set_secondary('chi2_checknimg 0', verb = 0)\n",
 "glafic.set_secondary('chi2_restart   -1', verb = 0)\n",
 "glafic.set_secondary('chi2_usemag    1', verb = 0)\n",
 "glafic.set_secondary('hvary          0', verb = 0)\n",
 "glafic.set_secondary('ran_seed -122000', verb = 0)\n",
 '\n',
 'glafic.startup_setnum(3)\n',
 'glafic.set_lens(1, "sie", 0.261343256161012, 157.9333, 0.02931999999999846, -0.021570000000000533, 0.1261772, -4.523281, 0.0, 0.0)\n',
 'glafic.set_lens(2, "sers", 0.261343256161012, 79706850000.0, 0.02931999999999846, -0.021570000000000533, 0.19174, 97.1557, 51.881, 1.4424)\n',
 'glafic.set_lens(3, "sers", 0.261343256161012, 9487091000.0, 0.02931999999999846, -0.021570000000000533, 0.58185, 80.9088, 5.97926, 0.5737)\n',
 '# glafic.set_lens

In [195]:
import re


# Read the existing file contents
with open('write_lens.py', 'r') as f:
    lines = f.readlines()

num_lens_profiles = len(df)
lens_name = (model_path + '/' + model_ver).strip("'\"")
print(f"Processing for {num_lens_profiles} lens profiles. Final state will be determined by this number.")

new_lines = []
# --- State variables to check if the lines exist at all ---
found_set_lens_4 = False
found_setopt_4 = False
last_lens_line_index = -1

for line in lines:
    # We use 'continue' to ensure each line is handled by only one block
    # and to prevent it from falling through to the 'else' at the end.

    # --- BLOCK 1: Handles all `glafic.set_lens(4)` cases ---
    if 'glafic.set_lens(4' in line:
        found_set_lens_4 = True
        if num_lens_profiles > 3:
            print("State: 4+ lenses. Ensuring 'glafic.set_lens(4)' is active and correct.")
            idx = 3 # This is the 4th lens
            lens_row = df.iloc[idx]
            model_name = str(lens_row["Lens Name"]).strip("'\"")
            lens_str = f'glafic.set_lens({idx+1}, "{model_name}", 0.261343256161012'
            for p in range(1, 8):
                lens_str += f', {lens_row[f"param{p}"]}'
            lens_str += ')\n'
            new_lines.append(lens_str)
        else:
            print("State: <=3 lenses. Ensuring 'glafic.set_lens(4)' is a commented placeholder.")
            new_lines.append('# glafic.set_lens(4)\n')
        last_lens_line_index = len(new_lines) - 1
        continue

    # --- BLOCK 2: Handles all `glafic.setopt_lens(4)` cases ---
    elif 'glafic.setopt_lens(4' in line:
        found_setopt_4 = True
        if num_lens_profiles > 3:
            print("State: 4+ lenses. Ensuring 'glafic.setopt_lens(4)' is active.")
            new_lines.append('glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n')
        else:
            print("State: <=3 lenses. Ensuring 'glafic.setopt_lens(4)' is a commented placeholder.")
            new_lines.append('# glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n')
        last_lens_line_index = len(new_lines) - 1
        continue

    # --- BLOCK 3: Handles general `glafic.set_lens` lines (1, 2, 3) ---
    elif 'glafic.set_lens(' in line:
        m = re.search(r'glafic\.set_lens\((\d+)', line)
        if m:
            idx = int(m.group(1)) - 1
            if idx < num_lens_profiles:
                lens_row = df.iloc[idx]
                model_name = str(lens_row["Lens Name"]).strip("'\"")
                lens_str = f'glafic.set_lens({idx+1}, "{model_name}", 0.261343256161012'
                for p in range(1, 8):
                    lens_str += f', {lens_row[f"param{p}"]}'
                lens_str += ')\n'
                print(f'Found and replaced glafic.set_lens({idx+1})')
                line = lens_str
        new_lines.append(line)
        last_lens_line_index = len(new_lines) - 1
        continue

    # --- FIXED BLOCK for glafic.set_point ---
    elif 'glafic.set_point(' in line:
        # Use a regular expression to safely capture the existing point ID
        m = re.search(r'glafic\.set_point\((\d+)', line)
        if m:
            point_id = int(m.group(1))
            # Reconstruct the line, preserving the original ID and using the correct values.
            # Assuming set_point_values = [val1, val2, val3]
            line = f'glafic.set_point({point_id}, 1.000, {set_point_values[1]}, {set_point_values[2]})\n'
            print(f"Found and replaced glafic.set_point({point_id})")
        
        new_lines.append(line)
        # CRITICAL: Add continue to prevent the line from being duplicated by the final 'else'
        continue
    
    # --- Other standard replacements ---
    elif 'glafic.startup_setnum(' in line:
        new_lines.append(f"glafic.startup_setnum({num_lens_profiles})\n")
    elif 'glafic.init' in line:
        new_lines.append(f"glafic.init(0.3, 0.7, -1.0, 0.7, '{lens_name}', -0.78, -0.78, 0.78, 0.78, 0.012, 0.012, 1, verb = 0)\n")
    else:
        # If none of the above match, add the line as is
        new_lines.append(line)

# --- After the loop, perform the self-healing check to add missing lines ---
if last_lens_line_index != -1:
    set_lens_4_placeholder = '# glafic.set_lens(4)\n'
    setopt_lens_4_placeholder = '# glafic.setopt_lens(4, 0, 0, 0, 0, 0, 0, 0, 0)\n'
    
    # We insert in reverse order to maintain correct positioning relative to the insertion point.
    if not found_setopt_4 and num_lens_profiles <= 3:
        print('Placeholder for "setopt_lens(4)" was missing entirely. Adding it back.')
        new_lines.insert(last_lens_line_index + 1, setopt_lens_4_placeholder)

    if not found_set_lens_4 and num_lens_profiles <= 3:
        print('Placeholder for "set_lens(4)" was missing entirely. Adding it back.')
        new_lines.insert(last_lens_line_index + 1, set_lens_4_placeholder)

# Write back once
with open('write_lens.py', 'w') as glafic_file:
    glafic_file.writelines(new_lines)

# --- Optional: Print the output file to verify ---
print("\n--- Content of updated write_lens.py ---")
with open('write_lens.py', 'r') as f:
    print(f.read())

Processing for 3 lens profiles. Final state will be determined by this number.
Found and replaced glafic.set_lens(1)
Found and replaced glafic.set_lens(2)
Found and replaced glafic.set_lens(3)
State: <=3 lenses. Ensuring 'glafic.set_lens(4)' is a commented placeholder.
Found and replaced glafic.set_point(1)
State: <=3 lenses. Ensuring 'glafic.setopt_lens(4)' is a commented placeholder.

--- Content of updated write_lens.py ---
#!/usr/bin/env python
import glafic

glafic.init(0.3, 0.7, -1.0, 0.7, 'Sersic/SIE/SIE_POS', -0.78, -0.78, 0.78, 0.78, 0.012, 0.012, 1, verb = 0)

glafic.set_secondary('chi2_splane 1', verb = 0)
glafic.set_secondary('chi2_checknimg 0', verb = 0)
glafic.set_secondary('chi2_restart   -1', verb = 0)
glafic.set_secondary('chi2_usemag    1', verb = 0)
glafic.set_secondary('hvary          0', verb = 0)
glafic.set_secondary('ran_seed -122000', verb = 0)

glafic.startup_setnum(3)
glafic.set_lens(1, "sie", 0.261343256161012, 157.9333, 0.02931999999999846, -0.02157000000000